In [1]:
# Install qubex library if not already installed
# !pip install git+https://github.com/amachino/qubex.git

In [2]:
# Import the necessary libraries
import numpy as np

from qubex.pulse import (
    Blank,
    Drag,
    FlatTop,
    Gaussian,
    Pulse,
    PulseSchedule,
    PulseSequence,
    VirtualZ,
)

In [ ]:
# Create a flat-top pulse
flattop = FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)

# Plot the pulse
flattop.plot()

# Print the pulse values
flattop.values

In [ ]:
# Scale the amplitude of the pulse
flattop.scaled(1.5).plot()

# Shift the phase of the pulse
flattop.shifted(np.pi).plot()

# Repeat the pulse
flattop.repeated(3).plot()

In [ ]:
# Create a Gaussian pulse
gaussian = Gaussian(
    duration=60,
    amplitude=0.5,
    sigma=10,
)

# Create a pulse sequence
seq = PulseSequence(
    [
        gaussian,
        Blank(10),
        gaussian.shifted(np.pi / 6),
        Blank(10),
        gaussian.scaled(1.5),
    ]
)

# Plot the pulse sequence
seq.plot()

# Print the pulse sequence values
seq.values

In [ ]:
# Create an arbitrary pulse
iq_array = [
    0.1 + 0.2j,
    0.7 + 0.3j,
    0.5 + 0.8j,
    0.2 + 0.7j,
    0.4 + 0.1j,
]
arbit = Pulse(iq_array)

# Plot the pulse
arbit.plot()

# Print the pulse values
arbit.values

In [ ]:
# Define channel labels
labels = ["Q00", "Q01", "Q02"]

# Create a pulse schedule for the channels
with PulseSchedule(labels) as ps:
    ps.add("Q00", flattop.repeated(5))
    ps.add("Q01", seq.scaled(2))
    ps.barrier()
    ps.add("Q00", arbit.repeated(3))
    ps.barrier(["Q00", "Q01"])
    ps.add("Q01", FlatTop(duration=100, amplitude=1, tau=10))
    ps.add("Q02", PulseSequence([gaussian, VirtualZ(np.pi / 2), gaussian]))

# Plot the pulse schedule
ps.plot()

# Get the PulseSequence dictionary
ps.get_sequences()

In [ ]:
# Get the sampled sequences
ps.get_sampled_sequences()

In [ ]:
# Create a pulse schedule using the previous pulse schedule

with PulseSchedule(labels) as ps2:
    ps2.call(ps)
    ps2.barrier()
    ps2.add("Q00", Blank(100))
    ps2.barrier()
    ps2.call(ps.scaled(-1))

ps2.plot()

In [ ]:
# Example: Cross-Resonance Sequence

cr_drive = FlatTop(
    duration=200,
    amplitude=1.0,
    tau=30,
)

drag_pi = Drag(
    duration=10,
    amplitude=0.5,
    beta=0.3,
)

targets = {
    "Control": {"frequency": 5, "object": "Q00"},
    "CR": {"frequency": 6, "object": "Q00"},
    "Target": {"frequency": 6, "object": "Q01"},
}

with PulseSchedule(targets) as cr:
    cr.add("CR", cr_drive)
    cr.add("Target", cr_drive.scaled(0.1).shifted(np.pi / 3))

with PulseSchedule(targets) as ecr:
    ecr.call(cr)
    ecr.add("Control", drag_pi)
    ecr.call(cr.scaled(-1))

ecr.plot(title="Echoed Cross-Resonance")